# 02d Emparejamiento

El "emparejamiento" (matching) es una nueva estructura de Python introducida en la versión 3.10. Comprueba que estás usando una versión de Python adecuada:

In [1]:
import sys
sys.version

'3.10.8 (main, Oct 12 2022, 00:00:00) [GCC 12.2.1 20220819 (Red Hat 12.2.1-2)]'

En su forma más sencilla, el emparejamiento funciona como una condicional múltiple, similar a `switch` en C/C++:

In [2]:
x = 5

match x:
    case 3:
        print("x vale 3")
    case 7:
        print("x vale 7")
    case _:
        print("x vale otro valor")

x vale otro valor


La cláusula `case _` es el equivalente a un `else`, emapreja con cualquier valor.

Podemos combinar varios valores con `|`, y el código se ejecutará si coincide con uno de ellos:

In [3]:
x = 5

match x:
    case 2 | 3 | 5 | 7:
        print("x es un número primo menor de 10")
    case _:
        print("No lo es")

x es un número primo menor de 10


Para tipos compuestos (incluyendo clases), podemos pedir que emparejen uno o varios de los elementos. Para los elementos que ignoramos, hay que usar `_`:

In [9]:
def posicion_punto(x):
    match x:
        case (0, 0):
            print("El punto es el origen")
        case (0, _):
            print("El punto está en el eje Y")
        case (_, 0):
            print("El punto está en el eje X")
        case (_, _):
            print("El punto no está en los ejes")
        case _:
            raise TypeError

In [10]:
posicion_punto((3, 0))
posicion_punto((0, -7))
posicion_punto((1, 8))
posicion_punto((0,0))
posicion_punto("a")

El punto está en el eje X
El punto está en el eje Y
El punto no está en los ejes
El punto es el origen


TypeError: 

`match` puede emparejar con valores que sean de un tipo determinado:

In [8]:
def primer_elemento(x):
    """Si x es un entero, lo devuelve, y si es una lista, devuelve el primer elemento"""
    match x:
        case float(x):
            return x
        case list(x):
            return x[0]
        case _:
            raise TypeError

In [7]:
print(primer_elemento(5.2))
print(primer_elemento([-3.7, 4.5]))

5.2
-3.7
